In [1]:
import requests
import json
import os
from bs4 import BeautifulSoup
import subprocess

In [2]:
def get_m3u8_video_urls(api_url):
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        m3u8_video_urls = [entry['VideoUrl'] for entry in data if entry.get('VideoUrl') and entry['VideoUrl'].endswith('.m3u8')]
        return m3u8_video_urls
    except Exception as e:
        print(f"Error: {e}")
        return []

# API URL
api_url = "https://511ny.org/api/getcameras?key=9fc53c6d396b4187b4e5bc3f00459ce0&format=json"

# Get the list of M3U8 video URLs
m3u8_video_urls = get_m3u8_video_urls(api_url)
print(len(m3u8_video_urls))

1512


In [ ]:
import os
import requests
import subprocess
from tqdm import tqdm  # Import tqdm for progress tracking

segments_folder = 'segments_folder'

if not os.path.exists(segments_folder):
    os.makedirs(segments_folder)

def is_m3u8_valid(m3u8_url):
    try:
        response = requests.get(m3u8_url)
        response.raise_for_status()  # Check for successful HTTP response
        if response.text.startswith('#EXTM3U'):
            return True
    except Exception as e:
        pass
    return False

def download_m3u8_to_mp4(m3u8_urls, output_folder, segment_duration=60, duration_limit=60):
    stream_ids = [str(i) for i in range(len(m3u8_urls))]
    skips = 0

    # Create a tqdm progress bar
    with tqdm(total=len(m3u8_urls)) as pbar:
        for url, stream_id in zip(m3u8_urls, stream_ids):
            output_filename = os.path.join(output_folder, f'stream_{stream_id}_segment_000.mp4')
            if os.path.exists(output_filename):
                skips += 1
            else:
                if is_m3u8_valid(url):
                    # ffmpeg command to download m3u8 stream and segment it into continuous MP4 files
                    cmd = [
                        'ffmpeg',
                        '-i', url,
                        '-c', 'copy',
                        '-t', str(duration_limit),
                        '-f', 'segment',
                        '-segment_time', str(segment_duration),
                        '-reset_timestamps', '1',
                        output_filename
                    ]
                    subprocess.call(cmd)
                else:
                    skips += 1
            pbar.update(1)  # Update the progress bar
    print(f"Successfully downloaded {len(m3u8_urls) - skips}\nSkipped {skips} streams.")

download_m3u8_to_mp4(m3u8_video_urls, segments_folder)

In [ ]:
for video in os.listdir(segments_folder):
    SOURCE_VIDEO_PATH = os.path.join(segments_folder, video)
    print(sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH))